## Soft prompting fine Tuning Demo

In [ ]:
# Cell 1: Setup TextGrad with LM Studio
import textgrad as tg
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

# Setup local LM Studio client
client = OpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio"  # LM Studio doesn't require real API key
)

LLM_MODEL = "qwen3-0.6b"

# Create custom TextGrad engine for LM Studio
class LMStudioEngine:
    def __init__(self, client, model):
        self.client = client
        self.model = model
    
    def __call__(self, prompt, system_prompt=None, **kwargs):
        messages = []
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        messages.append({"role": "user", "content": prompt})
        
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=0.1,
            max_tokens=1024
        )
        return response.choices[0].message.content

# Setup engines (using same model for both eval and target)
lm_engine = LMStudioEngine(client, LLM_MODEL)
tg.set_backward_engine(lm_engine, override=True)

print(f"Using model: {LLM_MODEL}")

# Cell 2: Create TextGrad Variables and Model
# System prompt as trainable variable
system_prompt = tg.Variable(
    "Create a recipe using the ingredients provided.",
    requires_grad=True,
    role_description="system prompt for recipe generation"
)

# Create model with trainable prompt
model = tg.BlackboxLLM(lm_engine, system_prompt)

# Test input
user_input = tg.Variable(
    "I want something for dinner and I have broccoli and cheese at home",
    requires_grad=False,
    role_description="user recipe request"
)

# Generate initial recipe
response = model(user_input)
print("Initial Recipe:")
print(response.value)

# Cell 3: Define Loss Function with Gemini Evaluation
import google.generativeai as genai

# Setup Gemini
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

genai.configure(api_key=GOOGLE_API_KEY)
gemini_model = genai.GenerativeModel("gemini-2.0-flash-exp")

def recipe_loss_fn(prediction, user_request):
    eval_prompt = f"""Rate this recipe from 1-10 (return ONLY the number):
- Uses available ingredients: broccoli and cheese
- Has clear cooking steps
- Is complete recipe

User: {user_request.value}
Recipe: {prediction.value}

Score:"""
    
    # Use Gemini for evaluation
    eval_response = gemini_model.generate_content(eval_prompt)
    
    score_text = eval_response.text.strip()
    try:
        # Extract number from response
        import re
        numbers = re.findall(r'\d+', score_text)
        if numbers:
            score = float(numbers[0])
            loss_value = f"Recipe score: {score}/10. Loss: {10-score}"
        else:
            loss_value = "Recipe needs improvement. Loss: 8"
    except:
        loss_value = "Recipe needs improvement. Loss: 8"
    
    return tg.Variable(loss_value, role_description="recipe quality assessment")

# Calculate loss
loss = recipe_loss_fn(response, user_input)
print(f"\nInitial Assessment: {loss.value}")

# Cell 4: Backward Pass and Update
# Create optimizer
optimizer = tg.TextualGradientDescent(engine=lm_engine, parameters=[system_prompt])
optimizer.zero_grad()

# Backward pass
loss.backward()

print("\nGradient (improvement suggestion):")
if system_prompt.gradients:
    print(system_prompt.gradients[0].value)

# Update the prompt
optimizer.step()

print(f"\nUpdated System Prompt:")
print(system_prompt.value)

# Cell 5: Multi-Epoch Training
# Training data
training_examples = [
    "I want dinner and I have broccoli and cheese at home",
    "I want lunch and I have pasta and tomatoes at home", 
    "I want breakfast and I have eggs and bread at home"
]

print("=== Training over multiple epochs ===")
for epoch in range(3):
    print(f"\nEpoch {epoch + 1}")
    epoch_losses = []
    
    for example in training_examples:
        # Create input variable
        input_var = tg.Variable(example, requires_grad=False, role_description="user request")
        
        # Generate response
        response = model(input_var)
        
        # Calculate loss
        loss = recipe_loss_fn(response, input_var)
        epoch_losses.append(loss)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f"  Example: '{example[:30]}...'")
        print(f"  Loss: {loss.value}")
    
    print(f"Current prompt: {system_prompt.value[:100]}...")

# Cell 6: Final Test
print("\n=== Final Test ===")
final_response = model(user_input)
print("Final Recipe:")
print(final_response.value)

final_loss = recipe_loss_fn(final_response, user_input)
print(f"\nFinal Assessment: {final_loss.value}")

Using model: qwen3-0.6b
Initial Recipe:
<think>
Okay, the user wants a dinner recipe with broccoli and cheese. Let me think about common dishes that pair well with these ingredients.

Broccoli is good in soups or stews, and cheese can be a good addition to any dish. Maybe a simple one-pan dish? Or maybe a more elaborate meal like a lasagna? Wait, but the user said they have broccoli and cheese at home, so perhaps a quick recipe that's easy to make.

A simple tomato-based soup with broccoli would work well. Let me check if that makes sense. Adding some herbs or olive oil could enhance it. Alternatively, maybe a baked dish like a cheese and broccoli casserole? That's a classic. But the user might prefer something more straightforward.

Wait, the user mentioned they have broccoli and cheese at home, so maybe they don't want to cook for someone else. Let me make sure the recipe is easy to prepare. A simple one-pan dish with some herbs would be good. Alternatively, a baked casserole with ch

c:\Users\TimPr\allcode\lazycook\lenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Initial Assessment: Recipe score: 5.0/10. Loss: 5.0

Gradient (improvement suggestion):

Updated System Prompt:
Create a recipe using the ingredients provided with detailed instructions and nutritional information.
=== Training over multiple epochs ===

Epoch 1
  Example: 'I want dinner and I have brocc...'
  Loss: Recipe score: 7.0/10. Loss: 3.0
  Example: 'I want lunch and I have pasta ...'
  Loss: Recipe score: 7.0/10. Loss: 3.0
  Example: 'I want breakfast and I have eg...'
  Loss: Recipe score: 8.0/10. Loss: 2.0
Current prompt: Create a recipe using the ingredients provided with detailed instructions and nutritional informatio...

Epoch 2
  Example: 'I want dinner and I have brocc...'
  Loss: Recipe score: 7.0/10. Loss: 3.0
  Example: 'I want lunch and I have pasta ...'
  Loss: Recipe score: 7.0/10. Loss: 3.0
  Example: 'I want breakfast and I have eg...'
  Loss: Recipe score: 7.0/10. Loss: 3.0
Current prompt: Craft a detailed recipe using the provided ingredients and step-by-ste